In [2]:
%matplotlib inline
import pandas as pd
import numpy as np

import fxtool as ft


In [3]:
def get_change_list(data, rate_type="Close"):
    '''
    Args:
        data: pd.DataFrame()
        rate_type: str
    Returns:
        list
    '''
    
    pre_rate = 1
    change_list = []
    for rate in data[rate_type]:
        change = np.log(rate/pre_rate)
        if change <= 1:
            change_list.append(change)
        else : 
            change_list.append(None)
        pre_rate = rate
    return change_list

In [4]:
data = ft.read_csv(["201506"])

201506


In [30]:
ohlc = ft.get_ohlc(data, 'H')
#ohlc = data 
change_list = get_change_list(ohlc)
ohlc["Change(Close)"] = change_list

ohlc['change_positive'] = 0
ohlc.ix[ohlc['Change(Close)'] >= 0, 'change_positive'] = 1
ohlc['change_negative'] = 0
ohlc.ix[ohlc['Change(Close)'] < 0, 'change_negative'] = 1

training_test_data = pd.DataFrame(
  columns=[
    'change_positive', 
    'change_negative',
    'change_1', 
    'change_2', 
    'change_3',
    ]
)

for i in range(7, len(ohlc)):
    change_positive = ohlc['change_positive'].ix[i]
    change_negative = ohlc['change_negative'].ix[i]
    change_1 = ohlc['Change(Close)'].ix[i-1]
    change_2 = ohlc['Change(Close)'].ix[i-2]
    change_3 = ohlc['Change(Close)'].ix[i-3]
    
    training_test_data = training_test_data.append(
        {
            'change_positive': change_positive,
            'change_negative': change_negative,
            'change_1': change_1,
            'change_2': change_2,
            'change_3': change_3
        },
        ignore_index=True
    )

training_test_data


predictors_tf = training_test_data[training_test_data.columns[2:]]

classes_tf = training_test_data[training_test_data.columns[:2]]

training_set_size = int(len(training_test_data) * 0.8)
test_set_size = len(training_test_data) - training_set_size

training_predictors_tf = predictors_tf[:training_set_size]
training_classes_tf = classes_tf[:training_set_size]
test_predictors_tf = predictors_tf[training_set_size:]
test_classes_tf = classes_tf[training_set_size:]


training_predictors_tf.describe()

,change_1,change_2,change_3
count,413.000000,413.000000,413.000000
mean,-0.000004,-0.000004,-0.000009
std,0.001208,0.001208,0.001206
min,-0.012475,-0.012475,-0.012475
25%,-0.000396,-0.000396,-0.000396
50%,0.000073,0.000073,0.000065
75%,0.000484,0.000484,0.000475
max,0.007165,0.007165,0.007165


In [36]:
import tensorflow as tf

sess1 = tf.Session()

num_predictors = len(training_predictors_tf.columns)
num_classes = len(training_classes_tf.columns)
print("num_predictor: {}".format( num_predictors))
print("num_classes: {}".format( num_classes))

feature_data = tf.placeholder("float", [None, num_predictors])
actual_classes = tf.placeholder("float", [None, num_classes])

weights1 = tf.Variable(tf.truncated_normal([num_predictors, 50], stddev=0.0001))
biases1 = tf.Variable(tf.ones([50]))

weights2 = tf.Variable(tf.truncated_normal([50, 25], stddev=0.0001))
biases2 = tf.Variable(tf.ones([25]))
                     
weights3 = tf.Variable(tf.truncated_normal([25, 2], stddev=0.0001))
biases3 = tf.Variable(tf.ones([2]))

# This time we introduce a single hidden layer into our model...
hidden_layer_1 = tf.nn.relu(tf.matmul(feature_data, weights1) + biases1)
hidden_layer_2 = tf.nn.relu(tf.matmul(hidden_layer_1, weights2) + biases2)
model = tf.nn.softmax(tf.matmul(hidden_layer_2, weights3) + biases3)

cost = -tf.reduce_sum(actual_classes*tf.log(model))

train_op1 = tf.train.AdamOptimizer(learning_rate=0.0001).minimize(cost)

init = tf.initialize_all_variables()
sess1.run(init)

num_predictor: 3
num_classes: 2
Tensor("Relu_3:0", shape=(?, 50), dtype=float32)


In [40]:
correct_prediction = tf.equal(tf.argmax(model, 1), tf.argmax(actual_classes, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))

for i in range(1, 300001):
  sess1.run(
    train_op1, 
    feed_dict={
      feature_data: training_predictors_tf.values, 
      actual_classes: training_classes_tf.values.reshape(len(training_classes_tf.values), 2)
    }
  )
  if i%5000 == 0:
    print( "{} :{}".format(i, sess1.run(
      accuracy,
      feed_dict={
        feature_data: training_predictors_tf.values, 
        actual_classes: training_classes_tf.values.reshape(len(training_classes_tf.values), 2)
      }
    )))

5000 :0.5714285969734192
10000 :0.5690072774887085
15000 :0.5762711763381958
20000 :0.5690072774887085
25000 :0.5714285969734192
30000 :0.5714285969734192
35000 :0.5714285969734192
40000 :0.5714285969734192
45000 :0.5714285969734192


KeyboardInterrupt: 